# The JuMP ecosystem for mathematical optimization

## JuliaCon 2018

## Juan Pablo Vielma
## MIT Sloan

## Minimum # of Passports to Visit all Countries?
[![Passport Index](img/passportindex.jpg "Passport Index")](https://www.passportindex.org)

199 passports = $10^{33}$ times the age of the universe to enumerate at $10^{17}$ flops!

In [2]:
# Download data from https://github.com/ilyankou/passport-index-dataset
#;git clone https://github.com/ilyankou/passport-index-dataset.git
data = readdlm(joinpath("passport-index-dataset","passport-index-country-names.csv"),',')
cntr = data[2:end,1]
vf = (x ->  x == -1 || x == 3 ? 1 :0).(data[2:end,2:end]);

## (Constrained) Mathematical Optimization and JuMP
$$
\begin{align*}
\min_{x,y} &&\quad \sum_{\operatorname{cntr} \;\in\; \operatorname{World}} \operatorname{pass}_{\,\operatorname{cntr}} \\
\text{s.t.}&&\quad  \operatorname{vf}(\operatorname{cntr},\operatorname{dst})*\operatorname{pass}_{\,\operatorname{cntr}} &\geq 1\quad  &\quad& \forall \; \operatorname{dst} \;\in \; \operatorname{World}\\
 &&\operatorname{pass}_{\,\operatorname{cntr}}  &\in \{0,1\}&\quad& \forall \; \operatorname{cntr}\in \; \operatorname{World}.
\end{align*}
$$

In [3]:
using JuMP, GLPK
model = Model(with_optimizer(GLPK.GLPKOptimizerMIP))
@variable(model, pass[1:length(cntr)], Bin)
@constraint(model, [j=1:length(cntr)], sum( vf[i,j]*pass[i] for i in 1:length(cntr)) >= 1)
@objective(model, Min, sum(pass))
JuMP.optimize(model)
print(JuMP.objectivevalue(model)," passports: ",join(cntr[find(JuMP.resultvalue.(pass))],", "))

24.0 passports: Afghanistan, Austria, Benin, Comoros, Djibouti, Eritrea, Gabon, Georgia, Hong Kong, India, Iraq, Libya, Madagascar, Maldives, New Zealand, North Korea, Papua New Guinea, Somalia, South Sudan, Sri Lanka, United Arab Emirates, United States, Viet Nam, Zimbabwe

## JuMP: Started by Students Leading a Vibrant Community





<center><img src="img/aa.svg" width="70%" style="margin:50px 0px"/></center>




# You Can Learn Optimization Using JuMP

<center><img src="img/Untitled 4.svg" width="110%" style="margin:100px -400px -400px -400px"/></center>

### You Can Develop New Methods Using Julia / JuMP

* Mixed-Integer Nonlinear Solvers from MIT and LANL:
    - Pajarito.jl, Juniper.jl, POD.jl and Katana.jl
* Porting experimental Interior Point Method from Matlab in ~ week:
    - 45x speed with linear algebra options
    - Already good out-of-the-box performance:

    
| Instance        | Matlab           | Julia   | Instance        | Matlab           | Julia  | Instance        | Matlab           | Julia   | Instance        | Matlab           | Julia  |
| ----------------|-----------------:|--------:|----------------:|-----------------:|-------:|----------------|-----------------:|--------:|----------------:|-----------------:|-------:|
| dense lp        | 5.8              | 4.1     | lotka-volt      |0.47              |0.38 | butcher         | 0.63             |    0.41 | reac-diff       | 0.32             |    0.23 |
| envelope        | 0.085            |   0.043 | motzkin         | 0.35             |    0.24 | caprasse        | 1.38             |    1.87 | robinson        | 0.34             |    0.23 |


    

# JuMP is Composable and Uses the Julia Ecosystem

[Drone Control Demo](https://github.com/juan-pablo-vielma/Dagstuhl-Seminar-18081/blob/master/Polynomial.ipynb) prepared in ~1 week:

<img src="img/drone.svg" width="80%" style="margin:0px 0px 0px 0px"/>

Running on Julia 0.6 / JuMP 0.18 (0.7/0.19 soon).


# JuMP is Evolving

* MOI

# Overview

# A step by step example
Let's see how we translate a simple, 2 variable LP to JuMP code.

$$
\begin{align*}
&\max_{x,y}& \quad x + 2y \\
&\text{s.t.}&\quad x + y &\leq 1 \\
&&0\leq x, y &\leq 1
\end{align*}
$$



Load JuMP, MathOptInterface (MOI), and GLPK (GNU LP/MIP solver):

In [34]:
using JuMP  
using MathOptInterface # Replaces MathProgBase
# shortcuts
const MOI = MathOptInterface
const MOIU = MathOptInterface.Utilities

using GLPK # Loading the GLPK module for using its solver

Construct a model object (a container for variables, constraints, solver options, etc.):

In [35]:
model = Model(with_optimizer(GLPK.GLPKOptimizerLP, msg_lev = 4));  
# Old syntax: model = Model(solver=GLPKSolverLP(msg_lev = 4)))

Define variables $0\leq x, y \leq 1$:

In [36]:
@variable(model, 0 <= x <= 1)
@variable(model, 0 <= y <= 1);

Add constraint $x + y \leq 1$:

In [37]:
@constraint(model, x + y <= 1)

x + y ≤ 1.0

Add objective $\max x + 2y$:

In [38]:
@objective(model, Max, x + 2y)

To solve the optimization problem, call the `optimize` function.

In [39]:
JuMP.optimize(model) # Old syntax: status = JuMP.solve(model)

GLPK Simplex Optimizer, v4.52
1 row, 2 columns, 2 non-zeros
Objective scale factor = -500
*     0: obj =   0.000000000e+00  infeas =  0.000e+00 (0)
*     2: obj =   2.000000000e+00  infeas =  0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND


We can then check the status of the optimization call.

In [40]:
@show JuMP.hasresultvalues(model)
@show JuMP.terminationstatus(model) == MOI.Success
@show JuMP.primalstatus(model) == MOI.FeasiblePoint
@show JuMP.dualstatus(model) == MOI.FeasiblePoint

true

JuMP.hasresultvalues(model) = true
JuMP.terminationstatus(model) == MOI.Success = true
JuMP.primalstatus(model) == MOI.FeasiblePoint = true
JuMP.dualstatus(model) == MOI.FeasiblePoint = true


# New Solver Status

Much more details than old `:Optimal, :Unbounded, :Infeasible, :UserLimit, :Error, :NotSolved`

```julia 
@show JuMP.terminationstatus(model) == MOI.Success
```

In [41]:
display(typeof(MOI.Success))

Enum MathOptInterface.TerminationStatusCode:
Success = 0
AlmostSuccess = 1
InfeasibleNoResult = 2
UnboundedNoResult = 3
InfeasibleOrUnbounded = 4
IterationLimit = 5
TimeLimit = 6
NodeLimit = 7
SolutionLimit = 8
MemoryLimit = 9
ObjectiveLimit = 10
NormLimit = 11
OtherLimit = 12
SlowProgress = 13
NumericalError = 14
InvalidModel = 15
InvalidOption = 16
Interrupted = 17
OtherError = 18

```julia
@show JuMP.primalstatus(model) == MOI.FeasiblePoint
```

In [42]:
display(typeof(MOI.FeasiblePoint))

Enum MathOptInterface.ResultStatusCode:
FeasiblePoint = 0
NearlyFeasiblePoint = 1
InfeasiblePoint = 2
InfeasibilityCertificate = 3
NearlyInfeasibilityCertificate = 4
ReductionCertificate = 5
NearlyReductionCertificate = 6
UnknownResultStatus = 7
OtherResultStatus = 8

We can also inspect the solution values and optimal cost:

In [43]:
@show JuMP.resultvalue(x)              # Old syntax: getvalue(x)
@show JuMP.resultvalue(y)              # Old syntax: getvalue(y)
@show JuMP.objectivevalue(model)       # Old syntax: getobjectivevalue(model)

2.0

JuMP.resultvalue(x) = -0.0
JuMP.resultvalue(y) = 1.0
JuMP.objectivevalue(model) = 2.0


I can also "name" constraints for later reference.

In [44]:
model = Model(with_optimizer(GLPK.GLPKOptimizerLP, msg_lev = 0))
@variable(model, 0 <= x <= 1)
@variable(model, 0 <= y <= 1)
@constraint(model, inequality, x + y <= 1)     # <=============== constraint can be referenced later as "inequality"
@objective(model, Max, x + 2y)
JuMP.optimize(model)
@show JuMP.terminationstatus(model) == MOI.Success

true

JuMP.terminationstatus(model) == MOI.Success = true


constraint references  can by used to delete them

In [45]:
JuMP.delete!(model, inequality)
JuMP.optimize(model)
@show JuMP.terminationstatus(model) == MOI.Success
@show JuMP.objectivevalue(model)  

3.0

JuMP.terminationstatus(model) == MOI.Success = true
JuMP.objectivevalue(model) = 3.0


Constraint references can be used to modify problem (see [MOI](MOI.ipynb#Model-modifications)) and to get duals (see [Topics notebook](Topics.ipynb#Duality)).

## Collections of variables/constraints and summation in JuMP

You can also create collections of variables like $x_i \geq 0 \quad \forall \; i\in\{1,\ldots,10\}$

In [49]:
model = Model()
@variable(model, x[1:10] >= 0);

Also multidimensional indexing, separated by commas:

In [50]:
@variable(model, y[1:10,["red","blue"]] <= 1);

and more complicated expressions like $\quad
i \leq z_{ij} \leq u_j \;\;\; \forall i \in \{1,...,10\}, j \in \{i+1, ..., 10\}
$:

In [51]:
u = rand(10)
@variable(model, i <= z[i=1:10,j=(i+1):10] <= u[j]);

To specify conditions on the indexing, you can add conditionals inside the ``[...]`` block, separated by a semicolon:

In [52]:
@variable(model, w[i=1:10, c=["red","blue"]; iseven(i) || c == "red"] >= 0)

Dict{Any,JuMP.VariableRef} with 15 entries:
  (10, "blue") => w[10,blue]
  (7, "red")   => w[7,red]
  (8, "red")   => w[8,red]
  (10, "red")  => w[10,red]
  (6, "blue")  => w[6,blue]
  (1, "red")   => w[1,red]
  (5, "red")   => w[5,red]
  (2, "blue")  => w[2,blue]
  (6, "red")   => w[6,red]
  (3, "red")   => w[3,red]
  (8, "blue")  => w[8,blue]
  (4, "red")   => w[4,red]
  (4, "blue")  => w[4,blue]
  (2, "red")   => w[2,red]
  (9, "red")   => w[9,red]

Also easy to create constrainta like $ \sum _{i = 1}^{10} x_i \leq 1$:

In [36]:
@constraint(model, sum(x[i] for i in 1:10) <= 1)

x[1] + x[2] + x[3] + x[4] + x[5] + x[6] + x[7] + x[8] + x[9] + x[10] ≤ 1.0

Or more complicated ones like 
$
\sum_{\substack{i\in\{1,...,10\}\\
                c\in\{"red","blue"\}}}
       coef(c) \cdot y_{ic} = 1
$

In [37]:
coef = Dict("red" => 2, "blue" => 3)
@constraint(model, sum(coef[c]*y[i,c] for i in 1:10, c in ["red","blue"]) == 1)

2 y[1,red] + 3 y[1,blue] + 2 y[2,red] + 3 y[2,blue] + 2 y[3,red] + 3 y[3,blue] + 2 y[4,red] + 3 y[4,blue] + 2 y[5,red] + 3 y[5,blue] + 2 y[6,red] + 3 y[6,blue] + 2 y[7,red] + 3 y[7,blue] + 2 y[8,red] + 3 y[8,blue] + 2 y[9,red] + 3 y[9,blue] + 2 y[10,red] + 3 y[10,blue] = 1.0

or $
\sum_{i = 1}^{10} \sum_{j = i+1}^{10} 
       i \cdot j \cdot z_{ij} \leq
\sum_{\substack{i\in\{1,...,10\},
                c\in\{"red","blue"\} \\
                \text{s.t. } iseven(i) \text{ or } c = "red"}}
       i^2 \cdot w_{ic} 
$:

In [38]:
@constraint(model, sum(i*j*z[i,j] for i in 1:10, j in (i+1):10) <=
               sum(i^2*w[i,c] for i in 1:10, c in ["red","blue"] if iseven(i) || c == "red"))

2 z[1,2] + 3 z[1,3] + 4 z[1,4] + 5 z[1,5] + 6 z[1,6] + 7 z[1,7] + 8 z[1,8] + 9 z[1,9] + 10 z[1,10] + 6 z[2,3] + 8 z[2,4] + 10 z[2,5] + 12 z[2,6] + 14 z[2,7] + 16 z[2,8] + 18 z[2,9] + 20 z[2,10] + 12 z[3,4] + 15 z[3,5] + 18 z[3,6] + 21 z[3,7] + 24 z[3,8] + 27 z[3,9] + 30 z[3,10] + 20 z[4,5] + 24 z[4,6] + 28 z[4,7] + 32 z[4,8] + 36 z[4,9] + 40 z[4,10] + 30 z[5,6] + 35 z[5,7] + 40 z[5,8] + 45 z[5,9] + 50 z[5,10] + 42 z[6,7] + 48 z[6,8] + 54 z[6,9] + 60 z[6,10] + 56 z[7,8] + 63 z[7,9] + 70 z[7,10] + 72 z[8,9] + 80 z[8,10] + 90 z[9,10] - w[1,red] - 4 w[2,red] - 4 w[2,blue] - 9 w[3,red] - 16 w[4,red] - 16 w[4,blue] - 25 w[5,red] - 36 w[6,red] - 36 w[6,blue] - 49 w[7,red] - 64 w[8,red] - 64 w[8,blue] - 81 w[9,red] - 100 w[10,red] - 100 w[10,blue] ≤ 0.0

Can also do collections of constraints (named or unamed):
    $$ 
\begin{align}
x_i &\leq 0.9 \quad \forall i \in \{1,2,3\} \quad\text{ (large bounds)}\\
x_i &\leq 0.5 \quad \forall i \in \{4,5,6\} 
\end{align}
$$

In [39]:
@constraint(model,largebounds[i=1:3], x[i] <= 0.9)
@constraint(model,[i=4:6], x[i] <= 0.5)

1-dimensional JuMPArray{JuMP.ConstraintRef{JuMP.Model,C} where C,1,...} with index sets:
    Dimension 1, 4:6
And data, a 3-element Array{JuMP.ConstraintRef{JuMP.Model,C} where C,1}:
 x[4] ≤ 0.5
 x[5] ≤ 0.5
 x[6] ≤ 0.5

# New in JuMP 0.91: New Containers and Conventions

`JuMPDict` is replaced by `Base.Dict` and `JuMPArray` was rewritten (inspired by `AxisArrays`). Conventions apply for `@variable`, `@constraint`, `@expression`, `@NLconstraint`, `@NLexpression`, ...

In [55]:
model = Model(with_optimizer(GLPK.GLPKOptimizerLP))
@variable(model, x[1:5, 1:5])            # Array
my_set = [:a, :b, :c]
@variable(model, w[1:5, my_set])        # JuMPArray
@variable(model, t[i = 1:5, 1:i])        # Dict
@variable(model, h[i = 1:5; isodd(i)]);  # Dict

Finally,  no more slicing error for JuMPArrays!

In [56]:
model = Model(with_optimizer(GLPK.GLPKOptimizerLP))
set_1 = [:a, :b, :c]
set_2 = [:x, :y, :z]
@variable(model, x[set_1,set_2])
x[:,:z]

1-dimensional JuMPArray{JuMP.VariableRef,1,...} with index sets:
    Dimension 1, Symbol[:a, :b, :c]
And data, a 3-element Array{JuMP.VariableRef,1}:
 x[a,z]
 x[b,z]
 x[c,z]

# A Warning on Performance and Type Stability 

`@variable` chooses the tightest applicable container while remaining **type stable**. 


In [58]:
model = Model(with_optimizer(GLPK.GLPKOptimizerLP))
@variable(model, x[1:5, 1:5])            # Array
set_1 = Base.OneTo(5)
@variable(model, y[set_1, 1:5])          # Array
set_2 = 1:5
@variable(model, z[1:5, set_2])          # JuMPArray
set_3 = [:a, :b, :c]
@variable(model, w[set_2, set_3])        # JuMPArray
@variable(model, t[i=set_2, 1:i])        # Dict
@variable(model, h[i = 1:5; isodd(i)]);  # Dict

You can also request a container type (for more details see [Internals](Internals.ipynb#JuMP-Containers) notebook):

In [59]:
model = Model(with_optimizer(GLPK.GLPKOptimizerLP))
@variable(model, x[1:5, 1:5], container = JuMPArray)
set_1 = 1:5
@variable(model, y[set_1, 1:5], container = Array)
set_2 = 2:3
# @variable(m, z[set_2, 1:5], container = Array)  # => Error instead of fallback to JuMPArray to preserve type stability

2:3

# Classes of Constraints Beyond Linear Inequalities

Broadcasted and two sided linear inequalities:

In [60]:
A = [1.0 2.0; 3.0 4.0]
l = [4.0, 5.0]
u = [2.0, 3.0]
model = Model()
@variable(model, x[1:2])
@constraint(model, l .<= A*x .<= u)

2-element Array{JuMP.ConstraintRef{JuMP.Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.Interval{Float64}}},1}:
 x[1] + 2 x[2] ∈ [4.0, 2.0]  
 3 x[1] + 4 x[2] ∈ [5.0, 3.0]

## Quadratic Inequalities:

Both convex:

In [63]:
model = Model()
@variable(model, x[1:2])
@constraint(model, x[1]^2 + x[2] <= 1)

x[1]² + x[2] ≤ 1.0

and non-convex:

In [64]:
@constraint(model, x[1]*x[2] - 1.0 == 0.0)

x[1]*x[2] = 1.0

## Conic constraints including...

Semidefinite constraints:

In [65]:
model = Model()                         # using CSDP; m = Model(optimizer = CSDP.CSDPOptimizer())
@variable(model, y[1:2,1:2], Symmetric)
@constraint(model, y in PSDCone())  
@variable(model, t)
@variable(model, w)
@SDconstraint(model,  [t 1; 1 -w] ⪰ [1 t; t -2])

[t - 1, -t + 1, -w + 2] ∈ MathOptInterface.PositiveSemidefiniteConeTriangle(2)

Second order cone constraints:
$$ 
\begin{equation}
\left\| Ax+u \right\|_2 \leq t
\end{equation}
$$

In [66]:
model = Model()
@variable(model, x[1:2])
@variable(model, t)
@constraint(model, [t;A*x+u] in SecondOrderCone())

[t, x[1] + 2 x[2] + 2, 3 x[1] + 4 x[2] + 3] ∈ MathOptInterface.SecondOrderCone(3)

Rotated second order cone constraints:
$$ 
\begin{equation}
\left\| Ax+u \right\|_2 \leq t \cdot w,\quad w\geq 0
\end{equation}
$$

In [61]:
model = Model()
@variable(model, x[1:2])
@variable(model, t)
@variable(model, w)
@constraint(model, [t;w;A*x+u] in RotatedSecondOrderCone())

[t, w, x[1] + 2 x[2] + 2, 3 x[1] + 4 x[2] + 3] ∈ MathOptInterface.RotatedSecondOrderCone(4)

# Also "derivavite based" nonlinear constraints

Remains unchanged:

In [67]:
model = Model()
@variable(model, x)
@variable(model, y)

@NLobjective(model, Min, (1-x)^2 + 100(y-x^2)^2)
@NLconstraint(model, exp(x)+sin(x) <=0)

(exp(x) + sin(x)) - 0.0 ≤ 0